# 使用 Milvus 和 DeepSeek 构建 RAG

检索 民法典 内容


## S1 准备环境

In [1]:
!pip install pymilvus -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [2]:
import os
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()
# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY")
# 获取 milvus的token
milvus_token = os.getenv("MILVUS_TOKEN")

print(api_key)
print(milvus_token)

sk-4aa800df99d44303a1d9e04e2b677f88
root:Ggi3teD22025allinai#


## S2 准备数据

In [19]:
 # 读取指定的单个文件
file_path = "./民法典.md"
with open(file_path, 'r', encoding='utf-8') as file:
        document = file.read()

text_lines = []
current_article = []
article_start_pattern = "**第"  # 条目标题起始模式

for line in document.split('\n'):
    stripped_line = line.strip()

    # 跳过完全空的行
    if not stripped_line:
        continue

    # 检查是否是条目标题行
    if stripped_line.startswith(article_start_pattern) and "条**" in stripped_line:
        # 如果已有累积的条目内容，保存到结果列表
        if current_article:
            text_lines.append('\n'.join(current_article))
            current_article = []

        # 开始新条目，添加标题行
        current_article.append(stripped_line)
        print(f"找到条目标题: {stripped_line[:30]}...")  # 调试输出
    else:
        # 非条目标题行，添加到当前条目内容
        if current_article:  # 确保已开始一个条目
            current_article.append(stripped_line)
            print(f"  添加内容行: {stripped_line[:30]}...")  # 调试输出

# 添加最后一个条目
if current_article:
    text_lines.append('\n'.join(current_article))

print(f"共识别出 {len(text_lines)} 个条目")  # 调试

找到条目标题: **第二百零四条** 为了明确物的归属，充分发挥物的效用，保...
找到条目标题: **第二百零五条** 本编调整因物的归属和利用产生的民事关系...
找到条目标题: **第二百零六条** 国家坚持和完善社会主义公有制为主体、多...
  添加内容行: 国家巩固和发展公有制经济，鼓励、支持和引导非公有制经济的发展...
  添加内容行: 国家实行社会主义市场经济，保障一切市场主体的平等法律地位和发...
找到条目标题: **第二百零七条** 国家、集体、私人的物权和其他权利人的物...
找到条目标题: **第二百零八条** 不动产权利的设立、变更、转让和消灭，应...
找到条目标题: **第二百零九条** 不动产物权的设立、变更、转让和消灭，经...
  添加内容行: 依法属于国家所有的自然资源，所有权可以不登记。...
找到条目标题: **第二百一十条** 不动产登记，由不动产所在地的登记机构办...
  添加内容行: 国家对不动产实行统一登记制度。统一登记的范围、登记机构和登记...
找到条目标题: **第二百一十一条** 当事人申请登记，应当根据不同登记事项...
  添加内容行: 申请登记材料以及登记事项相关信息，可以公开查询。...
找到条目标题: **第二百一十二条** 登记机构应当履行下列职责：...
  添加内容行: （一）审查申请人提供的材料；...
  添加内容行: （二）询问申请人；...
  添加内容行: （三）如实、及时登记；...
  添加内容行: （四）法律、行政法规规定的其他职责。...
  添加内容行: 申请登记的不动产存在尚未解决的权属争议的，登记机构应当不予登...
找到条目标题: **第二百一十三条** 登记机构不得有下列行为：...
  添加内容行: （一）要求对不动产进行评估；...
  添加内容行: （二）以不动产登记为条件收取其他费用；...
  添加内容行: （三）超出登记职责范围的其他行为。...
找到条目标题: **第二百一十四条** 不动产物权的设立、变更、转让和消灭，...
找到条目标题: **第二百一十五条** 不动产登记簿由登记机构管理。...
  添加内容行: 不动产登记簿应当采用纸质形式或者电子形式。...
  添加内容行: 不动产登记簿采用电子形式的，应当备份。...
找到条目标题:

## S3 准备 LLM 和 Embedding 模型

In [4]:
from openai import OpenAI

deepseek_client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)

In [6]:
from pymilvus import model as milvus_model

embedding_model = milvus_model.DefaultEmbeddingFunction()

test_embedding = embedding_model.encode_queries(["This is a test"])[0]
embedding_dim = len(test_embedding)
print(embedding_dim)

768


## S4 将数据加载到 Milvus

In [20]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(uri="http://c-6314c3df7e880376.milvus.aliyuncs.com:19530",
                             token=milvus_token,
                             db_name="rag_demo01")

collection_name = "mfd_collection02"

# 检查 collection 是否已存在，如果存在则删除它
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="IP",
    consistency_level="Strong"
)

In [21]:
# 插入数据
from tqdm import tqdm

data = []

doc_embeddings = embedding_model.encode_documents(text_lines)

for i, line in enumerate(tqdm(text_lines, desc="Creating embeddings")):
    data.append({"id": i, "vector": doc_embeddings[i], "text": line})

milvus_client.insert(collection_name=collection_name, data=data)

Creating embeddings: 100%|██████████| 387/387 [00:00<00:00, 647982.29it/s]


{'insert_count': 387, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 

## S5 构建 RAG

In [24]:
# 检索
question = "合同不生效、无效、被撤销或者终止，要看哪条法律?"

search_res = milvus_client.search(
    collection_name=collection_name,
    data=embedding_model.encode_queries(
        [question]
    ),  # 将问题转换为嵌入向量
    limit=3,  # 返回前3个结果
    search_params={"metric_type": "IP", "params": {}},  # 内积距离
    output_fields=["text"],  # 返回 text 字段
)

In [26]:
# 输出检索结果
import json

retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4))

[
    [
        "**\u7b2c\u4e8c\u767e\u4e8c\u5341\u4e09\u6761** \u52a8\u4ea7\u7269\u6743\u7684\u8bbe\u7acb\u548c\u8f6c\u8ba9\uff0c\u81ea\u4ea4\u4ed8\u65f6\u53d1\u751f\u6548\u529b\uff0c\u4f46\u662f\u6cd5\u5f8b\u53e6\u6709\u89c4\u5b9a\u7684\u9664\u5916\u3002",
        0.6877487301826477
    ],
    [
        "**\u7b2c\u4e8c\u767e\u4e00\u5341\u56db\u6761** \u4e0d\u52a8\u4ea7\u7269\u6743\u7684\u8bbe\u7acb\u3001\u53d8\u66f4\u3001\u8f6c\u8ba9\u548c\u6d88\u706d\uff0c\u4f9d\u7167\u6cd5\u5f8b\u89c4\u5b9a\u5e94\u5f53\u767b\u8bb0\u7684\uff0c\u81ea\u8bb0\u8f7d\u4e8e\u4e0d\u52a8\u4ea7\u767b\u8bb0\u7c3f\u65f6\u53d1\u751f\u6548\u529b\u3002",
        0.6877487301826477
    ],
    [
        "**\u7b2c\u4e8c\u767e\u96f6\u516b\u6761** \u4e0d\u52a8\u4ea7\u6743\u5229\u7684\u8bbe\u7acb\u3001\u53d8\u66f4\u3001\u8f6c\u8ba9\u548c\u6d88\u706d\uff0c\u5e94\u5f53\u4f9d\u7167\u6cd5\u5f8b\u89c4\u5b9a\u767b\u8bb0\u3002\u52a8\u4ea7\u7269\u6743\u7684\u8bbe\u7acb\u548c\u8f6c\u8ba9\uff0c\u5e94\u5f53\u4f9d\u7167\u6cd5\u5f8b

In [27]:
# 使用LLM获取RAG响应
print('question : ' + question)

context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)
print('检索内容转换为 字符串格式 : ' + context)

SYSTEM_PROMPT = """
Human: 你是一个 专业的法律顾问。你能够从提供的上下文段落片段中找到问题的答案。
"""
USER_PROMPT = f"""
请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加原始回答的中文翻译，并用 <translated>和</translated> 标签标注。
<context>
{context}
</context>
<question>
{question}
</question>
<translated>
</translated>
"""

print('prompt内容 : ' + USER_PROMPT)

question : 合同不生效、无效、被撤销或者终止，要看哪条法律?
检索内容转换为 字符串格式 : **第二百二十三条** 动产物权的设立和转让，自交付时发生效力，但是法律另有规定的除外。
**第二百一十四条** 不动产物权的设立、变更、转让和消灭，依照法律规定应当登记的，自记载于不动产登记簿时发生效力。
**第二百零八条** 不动产权利的设立、变更、转让和消灭，应当依照法律规定登记。动产物权的设立和转让，应当依照法律规定交付。
prompt内容 : 
请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加原始回答的中文翻译，并用 <translated>和</translated> 标签标注。
<context>
**第二百二十三条** 动产物权的设立和转让，自交付时发生效力，但是法律另有规定的除外。
**第二百一十四条** 不动产物权的设立、变更、转让和消灭，依照法律规定应当登记的，自记载于不动产登记簿时发生效力。
**第二百零八条** 不动产权利的设立、变更、转让和消灭，应当依照法律规定登记。动产物权的设立和转让，应当依照法律规定交付。
</context>
<question>
合同不生效、无效、被撤销或者终止，要看哪条法律?
</question>
<translated>
</translated>



In [28]:
# 使用 DeepSeek 提供的 `deepseek-chat` 模型根据提示生成响应。
response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
print(response.choices[0].message.content)

根据提供的上下文，您询问的"合同不生效、无效、被撤销或者终止"的情形并未在给出的法条中直接规定。给出的第二百二十三条、第二百一十四条和第二百零八条主要涉及的是物权变动（包括动产和不动产）的效力规则，而非合同效力问题。

关于合同效力问题（包括不生效、无效、被撤销或终止），通常应参考《中华人民共和国民法典》合同编的相关规定，特别是：
- 合同效力瑕疵：第144条（无民事行为能力）、第146条（虚假表示）、第153条（违法背俗）等
- 合同撤销：第147-151条（重大误解、欺诈、胁迫等）
- 合同终止：第557条等

建议查阅民法典合同编（第三编）第一章"合同的订立"和第五章"合同的保全和变更转让"等相关条款。

<translated>
根据提供的上下文，您询问的"合同不生效、无效、被撤销或者终止"的情形并未在给出的法条中直接规定。给出的第223条、第214条和第208条主要涉及的是物权变动（包括动产和不动产）的效力规则，而非合同效力问题。

关于合同效力问题（包括不生效、无效、被撤销或终止），通常应参考《中华人民共和国民法典》合同编的相关规定，特别是：
- 合同效力瑕疵：第144条（无民事行为能力）、第146条（虚假表示）、第153条（违法背俗）等
- 合同撤销：第147-151条（重大误解、欺诈、胁迫等）
- 合同终止：第557条等

建议查阅民法典合同编（第三编）第一章"合同的订立"和第五章"合同的保全和变更转让"等相关条款。
</translated>
